In [9]:
import pandas as pd
import serial
import re
import math
import time

In [ ]:
def text2cor(inn: str) -> float:
    try:
        infloat = float(inn)
    except ValueError:
        return 0
    else:
        infloat /= 100
        infloat = int(infloat) + (infloat - int(infloat)) / 60 * 100
        return infloat


def cor(curr_latitude: str, curr_longitude: str) -> tuple:
    return text2cor(curr_latitude), text2cor(curr_longitude)


def get_len(x, y):
    return math.sqrt(x * x + y * y)


def get_ang(x: float, y: float) -> any:
    length = get_len(x, y)
    if length == 0:
        return None
    ang = math.atan2(x, -y) * 180 / math.pi
    return (ang + 270) % 360


def navigate(lat_cur: float, long_cur: float, lat_goal: float, long_goal: float) -> tuple:
    d_lat = lat_goal - lat_cur
    d_long = long_goal - long_cur
    lat_dist = d_lat * dist_1deg
    long_dist = d_long * dist_1deg * math.cos(lat_cur)
    dist = get_len(lat_dist, long_dist)
    direct = get_ang(lat_dist, long_dist)
#    print(dist, direct)
    return direct, dist


def move(goal_ang: float, curr_ang: float, dist: float):
    if dist > STOP_DIST:
        if ((goal_ang - curr_ang + 360) % 360) < SEARCH_ANG or ((goal_ang - curr_ang + 360) % 360) > (360 + SEARCH_ANG):
            print("BOTH MOTORS!")
        elif ((goal_ang - curr_ang + 360) % 360) < ((curr_ang - goal_ang + 360) % 360):
            print("LEFT MOTOR!")
        elif ((goal_ang - curr_ang + 360) % 360) >= ((curr_ang - goal_ang + 360) % 360):
            print("RIGHT MOTOR!")
    else:
        print('STOP!')

In [ ]:
flag_gps = False

df = pd.DataFrame(columns=['X', 'Y', 'Z'])

while 1:
    if flag_gps:
        try:
            while ser_gps.inWaiting():
                input_gps_list = ser_gps.readlines(ser_gps.inWaiting())
                for i in input_gps_list:
                    i = str(i)
                    i = i.split(',')
                    if str.find(i[0], 'GPGGA') != -1:
                        lat, long = cor(i[2], i[4])
                        print(lat, long)
                    elif str.find(i[0], 'GNGLL') != -1:
                        lat, long = cor(i[1], i[3])
                        print(lat, long)
                    elif str.find(i[0], 'GNRMC') != -1:
                        lat, long = cor(i[3], i[5])
                        print(lat, long)
        except OSError:
            print('Error COM10')
            flag_gps = False
    else:
        try:
            ser_gps = serial.Serial('COM10', 4800)
        except OSError:
            flag_gps = False
            print('COM10 is not found')
            time.sleep(1)
        else:
            flag_gps = True
    # try:
    #     direct, dist = navigate(lat, long, LAT_GOAL, LONG_GOAL)
    #     #print(direct, dist)
    #     move(direct, curr_angle, dist)
    #     print(dist*1000)
    #     print(lat, lat_goal, long, long_goal)
    # except NameError:
    #     continue

# time.sleep(0.2)

# df.to_csv(r'D:\data_calibration2.csv', index = False)
# 51.9 105.06510